# 1.Preparation

Connect to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change this to your own address.

In [ ]:
!ls "/content/drive/MyDrive/Your own address"

 Add_Clustered_news_0.6				     HON_2023-10-30_2024-09-30.json
'Benchmark with Llama3 8b Data.ipynb'		     KO_2023-10-30_2024-09-30.json
 Clustered_news_0.6				     MCD_2023-10-30_2024-09-30.json
 config.json					    'News repetition reduction.ipynb'
 dataset_dict.json				     __pycache__
 DIS_2023-10-30_2024-09-30.json			     test
 finetuned_models				     test_demo.ipynb
 fingpt-forecaster				     train
 fingpt-forecaster-1029				     train_lora.py
 fingpt-forecaster-1102				     train.sh
 fingpt-forecaster-1105				     Untitled
 fingpt-forecaster-dow-30-20230930-20240930-1-4-06   utils.py
 fingpt-forecaster-dow-30-20231030-20240930-1-4-06   wandb
 HD_2023-10-30_2024-09-30.json


In [ ]:
%cd "/content/drive/MyDrive/Your own address"

/content/drive/MyDrive/Colab Notebooks/AI4Finance/FinForecaster/Benchmark with Llama3 8b Data


In [ ]:
!pip install transformers torch datasets peft wandb
!pip install deepspeed
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA A100-SXM4-40GB


# 2.Data Loading and Formatting

In [1]:
from datasets import load_from_disk


train_dataset = load_from_disk('/content/drive/MyDrive/Your Trainging data address/')

ModuleNotFoundError: No module named 'datasets'

In [ ]:
def get_unique_ticker_symbols(test_dataset):

    ticker_symbols = set()


    for i in range(len(test_dataset)):
        prompt_content = test_dataset[i]['prompt']


        ticker_symbol = re.search(r"ticker\s([A-Z]+)", prompt_content)


        if ticker_symbol:
            ticker_symbols.add(ticker_symbol.group(1))


    return list(ticker_symbols)



def insert_guidance_after_intro(prompt):


    intro_marker = (
        "[INST]<<SYS>>\n"
        "You are a seasoned stock market analyst. Your task is to list the positive developments and "
        "potential concerns for companies based on relevant news and basic financials from the past weeks, "
        "then provide an analysis and prediction for the companies' stock price movement for the upcoming week."
    )
    guidance_start_marker = "Based on all the information before"
    guidance_end_marker = "Following these instructions, please come up with 2-4 most important positive factors"


    intro_pos = prompt.find(intro_marker)
    guidance_start_pos = prompt.find(guidance_start_marker)
    guidance_end_pos = prompt.find(guidance_end_marker)


    if intro_pos == -1 or guidance_start_pos == -1 or guidance_end_pos == -1:
        return prompt


    guidance_section = prompt[guidance_start_pos:guidance_end_pos].strip()


    new_prompt = (
        f"{prompt[:intro_pos + len(intro_marker)]}\n\n"
        f"{guidance_section}\n\n"
        f"{prompt[intro_pos + len(intro_marker):guidance_start_pos]}"
        f"{prompt[guidance_end_pos:]}"
    )

    return new_prompt


def apply_to_all_prompts_in_dataset(test_dataset):


    updated_dataset = test_dataset.map(lambda x: {"prompt": insert_guidance_after_intro(x["prompt"])})

    return updated_dataset

In [ ]:
train_dataset = apply_to_all_prompts_in_dataset(train_dataset)

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

Check the trining data

In [ ]:
train_dataset['prompt'][0]

"[INST]<<SYS>>\nYou are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ...\n\n<</SYS>>\n\n[Company Introduction]:\n\nMicrosoft Corp is a leading entity in the Technology sector. Incorporated and publicly traded since 1986-03-13, the company has established its reputation as one of the key players in the market. As of today, Microsoft Corp has a market capitalization of 3051052.10 in USD, with 7433.04 shares outstanding.\n\nMicrosoft Corp operates primarily in the US, trading under the ticker MSFT on the NASDAQ NMS - GLOBAL MARKET. As a dominant force in the Technology space, the company con

In [ ]:
from huggingface_hub import login

# 输入你的 Hugging Face Access Token
login(token="Use your own key to make the call for the base model")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 3.Executive training document

In [ ]:
!bash train.sh

[2024-11-05 20:23:36,145] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-05 20:23:42,089] [WARNING] [runner.py:215:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-11-05 20:23:42,089] [INFO] [runner.py:607:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None train_lora.py --run_name llama3-8b-a100-5e-5lr --base_model llama3 --dataset /content/drive/MyDrive/Colab Notebooks/AI4Finance/FinForecaster/Benchmark with Llama3 8b Data/fingpt-forecaster-1105/train/ --test_dataset /content/drive/MyDrive/Colab Notebooks/AI4Finance/FinForecaster/Benchmark with Llama3 8b Data/fingpt-forecaster-1105/test/ --max_length 8000 --batch_size 2 --gradient_accumulation_steps 8 --learning_rate 5e-5 --num_epochs 5 --log_interval 10 --warmup_ratio 0.03 --scheduler constant --evaluation_str